In [2]:
import os
import cv2
from tqdm import tqdm
import albumentations as A
import random

def augment_and_save_noise(src_root, dest_root, transform):
    # Ensure the training and testing directories exist
    train_root = os.path.join(dest_root, 'train')
    test_root = os.path.join(dest_root, 'test')

    if not os.path.exists(train_root):
        os.makedirs(train_root)
    if not os.path.exists(test_root):
        os.makedirs(test_root)

    # List all class directories in the source root
    class_names = [d for d in os.listdir(src_root) if os.path.isdir(os.path.join(src_root, d))]

    for class_name in tqdm(class_names):
        # Ensure the class directory exists in both the training and testing directories
        dest_train_class_dir = os.path.join(train_root, class_name)
        dest_test_class_dir = os.path.join(test_root, class_name)

        if not os.path.exists(dest_train_class_dir):
            os.makedirs(dest_train_class_dir)
        if not os.path.exists(dest_test_class_dir):
            os.makedirs(dest_test_class_dir)

        src_class_dir = os.path.join(src_root, class_name)
        image_names = [img for img in os.listdir(src_class_dir) if img.endswith(('.jpg', '.png'))]
        random.shuffle(image_names)  # Shuffle image names for random split

        # Calculate the split index for training and testing sets
        split_idx = int(0.8 * len(image_names))
        train_images = image_names[:split_idx]
        test_images = image_names[split_idx:]

        # Save the augmented images
        for image_name, dest_class_dir in zip([train_images, test_images], [dest_train_class_dir, dest_test_class_dir]):
            for img in image_name:
                image_path = os.path.join(src_class_dir, img)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

                augmented = transform(image=image)
                augmented_image = augmented['image']
                img = img.replace(".png", "")

                dest_image_path = os.path.join(dest_class_dir, img)
                cv2.imwrite(f"{dest_image_path}-noise.png", cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving

# Define the albumentations transformations you want
transform = A.Compose([
    A.Resize(height=64, width=64),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(var_limit=(10, 100), p=1)
    # Add any other augmentations as needed
])

src_directory = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train/'
dest_directory = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train-adj-train-test/'

augment_and_save_noise(src_directory, dest_directory, transform)


100%|██████████| 12/12 [00:21<00:00,  1.76s/it]


In [ ]:
#
#   NO ADJUSTMENT FOR TEST DATA
#

import os
import cv2
from tqdm import tqdm
import albumentations as A
import random
import shutil

def augment_and_save_noise(src_root, dest_root, transform):
    # Ensure the training and testing directories exist
    train_root = os.path.join(dest_root, 'train')
    test_root = os.path.join(dest_root, 'test')

    if not os.path.exists(train_root):
        os.makedirs(train_root)
    if not os.path.exists(test_root):
        os.makedirs(test_root)

    # List all class directories in the source root
    class_names = [d for d in os.listdir(src_root) if os.path.isdir(os.path.join(src_root, d))]

    for class_name in tqdm(class_names):
        # Ensure the class directory exists in both the training and testing directories
        dest_train_class_dir = os.path.join(train_root, class_name)
        dest_test_class_dir = os.path.join(test_root, class_name)

        if not os.path.exists(dest_train_class_dir):
            os.makedirs(dest_train_class_dir)
        if not os.path.exists(dest_test_class_dir):
            os.makedirs(dest_test_class_dir)

        src_class_dir = os.path.join(src_root, class_name)
        image_names = [img for img in os.listdir(src_class_dir) if img.endswith(('.jpg', '.png'))]
        random.shuffle(image_names)  # Shuffle image names for random split

        # Calculate the split index for training and testing sets
        split_idx = int(0.8 * len(image_names))
        train_images = image_names[:split_idx]
        test_images = image_names[split_idx:]

        # Directly copy training images without any transformation
        for img in train_images:
            src_image_path = os.path.join(src_class_dir, img)
            dest_image_path = os.path.join(dest_train_class_dir, img)
            shutil.copy(src_image_path, dest_image_path)

        # Save the augmented test images
        for img in test_images:
            image_path = os.path.join(src_class_dir, img)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

            augmented = transform(image=image)
            augmented_image = augmented['image']
            img = img.replace(".png", "")

            dest_image_path = os.path.join(dest_test_class_dir, img)
            cv2.imwrite(f"{dest_image_path}-noise.png", cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving

# Define the albumentations transformations you want
transform = A.Compose([
    A.Resize(height=64, width=64),
    A.RandomBrightnessContrast(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, rotate_limit=0.1, scale_limit=0, p=0.5),
    A.GaussNoise(var_limit=(10, 200), p=0.6)
    # Add any other augmentations as needed
])

src_directory = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train/'
dest_directory = 'C:/Users/denis/Desktop/HNS/projekt1/Dataset_Znacky/Train-adj-train-test/'

augment_and_save_noise(src_directory, dest_directory, transform)